In [2]:
import tensorflow as tf
# import tf.keras as keras

In [3]:
from tensorflow import keras

In [4]:

import dlib

In [5]:
import cv2

In [6]:
import mtcnn

In [7]:
from mtcnn.mtcnn import MTCNN

In [8]:
model = MTCNN()

In [292]:
imag = cv2.imread('./myImage2.jpeg')

In [293]:
cv2.imshow('image',imag)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [294]:
from PIL import Image as im
import numpy as np
import matplotlib.pyplot as plt


In [295]:
def normalise(cropped_image,original_image,face_with_landmarks, desired_left_eye=(0.33,0.33), desired_nose=(0.5,0.5)):
    
    faceX, faceY, face_width, face_height = face_with_landmarks['box']
    left_eye = np.array(face_with_landmarks['keypoints']['left_eye'])
    right_eye = np.array(face_with_landmarks['keypoints']['right_eye'])
    eye_center = (left_eye + right_eye)/2
    
    nose = np.array(face_with_landmarks['keypoints']['nose'])
    nose_distance = np.sum((eye_center - nose)**2)
    print("nose_distance ",nose_distance)
    eyes_distance = np.sum((left_eye - right_eye)**2)
    print("eyes_distance ",eyes_distance)
    input_points = np.float32([left_eye, right_eye, nose])

    scaling_factor = nose_distance/eyes_distance 
    print("scaling factor ",scaling_factor)
    desired_nose = np.array([0.5,2*desired_left_eye[1]-(1-2*desired_left_eye[0])*scaling_factor])
    print("desired nose ",desired_nose)

    desired_right_eye = np.array([1-desired_left_eye[0],desired_left_eye[1]])*np.array([face_width,face_height])
    desired_left_eye = np.array(desired_left_eye)*np.array([face_width,face_height])

    # desired_nose = np.array(desired_nose)*np.array([face_width,face_height])
    desired_nose = desired_nose*np.array([face_width,face_height])

    output_points = np.float32([desired_left_eye, desired_right_eye, desired_nose]) 

    M = cv2.getAffineTransform(input_points, output_points)
    
    dst = cv2.warpAffine(original_image, M,(face_width,face_height))
    return dst 




In [296]:
faces = model.detect_faces(imag)
print(faces)
cv2.imshow("cropped image",imag)
cv2.waitKey(0)
cv2.destroyAllWindows()


[{'box': [277, 457, 264, 336], 'confidence': 0.9893486499786377, 'keypoints': {'left_eye': (345, 567), 'right_eye': (468, 585), 'nose': (384, 638), 'mouth_left': (332, 703), 'mouth_right': (431, 722)}}]


In [297]:

for face in faces:
    print("faces detected")
    x,y,width,height =face['box']
    print(face["box"])
    landmarks = face['keypoints']
    leye = landmarks['left_eye']
    reye = landmarks['right_eye']
    print(leye)
    print(reye)
 
    cv2.imshow("cropped image",imag)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    newImage = imag[y:(y+height),x:(x+width)]
    cv2.imshow("cropped image",newImage)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # newImage= facealigner(newImage,(x,y), leye, reye, height, width, (400,400),(0.2,0.2))
    newImage =normalise(newImage,imag,face)
    print(newImage.shape)
    cv2.imshow("cropped image",newImage)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

faces detected
[277, 457, 264, 336]
(345, 567)
(468, 585)
nose_distance  4350.25
eyes_distance  15453
scaling factor  0.28151491619750213
desired nose  [0.5        0.56428493]
(336, 264, 3)


In [11]:
x,y,width,height =faces[0]['box']

In [19]:
rect = cv2.rectangle(imag,(x,y),(x+width,y+height),(0,0,255))
landmarks=faces[0]['keypoints']
for i in landmarks:
    rect=cv2.circle(rect,landmarks[i],3,(0,0,255))
cv2.imshow("image",rect)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [17]:
landmarks=faces[0]['keypoints']
for i in landmarks:
    print(landmarks[i])

(391, 559)
(570, 562)
(477, 658)
(398, 745)
(548, 750)


In [36]:
# vid=cv2.VideoCapture(0)
# processFrame = 0
# while(True):
#     ret,frame = vid.read()
#     # print(frame.shape)
#     if processFrame%2 == 0:
#         faces = model.detect_faces(frame)
#         for face in faces:
#             box = face['box']
#             frame = cv2.rectangle(frame,(box[0],box[1]),(box[0]+box[2],box[1]+box[3]),(0,0,255))
#             landmarks = face['keypoints']
#             for i in landmarks:
#                 frame=cv2.circle(frame,landmarks[i],3,(0,0,255))
#         cv2.imshow('detection',frame)
#     processFrame = processFrame + 1
#     if cv2.waitKey(1) & 0xFF ==ord('q'):
#         break

# vid.release()
# cv2.destroyAllWindows()

In [ ]:
vid=cv2.VideoCapture(0)
processFrame = 0
while(True):
    ret,frame = vid.read()
    # print(frame.shape)
    if processFrame%2 == 0:
        faces = model.detect_faces(frame)
        for face in faces:
            croppedFace = faces[]
    processFrame = processFrame + 1
    if cv2.waitKey(1) & 0xFF ==ord('q'):
        break

vid.release()
cv2.destroyAllWindows()